<a href="https://colab.research.google.com/github/stewartjollymore/MNIST_NNs/blob/main/Numpy%20NNs/Fully_Connected_FullTrainingLoop_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.datasets import mnist
from keras.utils import to_categorical


#Below we separate and arange the data as seen around the web
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
x_train /= 255
# encode output which is a number in range [0,9] into a vector of size 10
# e.g. number 3 will become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
y_train = to_categorical(y_train)

# same for test data : 10000 samples
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
x_test /= 255
y_test = to_categorical(y_test)

11490434/11490434 [==============================] - 0s 0us/step


In [82]:
#Creating the layer dimensions

input_size1 = 28*28
output_size1 = 100
input_size2 = 100
output_size2 = 50
input_sizefinal = 50
output_sizefinal = 10

In [219]:
import numpy as np

#Initialze weights and biases

w1 = np.random.rand(input_size1, output_size1) - 0.5
b1 = np.random.rand(1, output_size1) - 0.5

w2 = np.random.rand(input_size2, output_size2) - 0.5
b2 = np.random.rand(1, output_size2) - 0.5

w3 = np.random.rand(input_sizefinal, output_sizefinal) -0.5
b3 = np.random.rand(1, output_sizefinal) - 0.5

In [208]:
import tensorflow as tf
import keras as ks

#Define activations for forward prop and backprop

def tanh(x):
    return np.tanh(x);

def gradient(x):
    return 1-np.tanh(x)**2;

In [209]:
#define the error metrics

def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_grad(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

In [248]:
samples = len(x_train)
epochs = 2

# training loop
for i in range(epochs):
  err = 0
  for j in range(samples):
  # forward propagation
    single_xtrain = x_train[j]
    single_ytrain = y_train[j]

    #Pass through first hidden layer and activation neruon
    layer1 = np.dot(single_xtrain,w1)+b1
    act_layer1 = tanh(layer1)

    #Pass through second hidden layer and activation neuron
    layer2 = np.dot(act_layer1,w2)+b2
    act_layer2 = tanh(layer2)

    #Pass through output layer and acrivation neuron
    layer3 = np.dot(act_layer2,w3)+b3
    output = tanh(layer3)

    err += mse(single_ytrain, output)
    #print(single_xtrain)
    #Backward Propigation steps

    #Define batch size and learning rate
    N = single_ytrain.size
    learning_rate = 0.01

    #calculate the output error
    #output_error = mse_grad(single_ytrain,output)
    output_error = single_ytrain - output

    #Backprop through output layer
    output_grad = output_error*gradient(output)
    w3_grad = np.dot(layer2.T, output_grad)
    b3_grad = np.sum(output_grad)
    layer2_grad = np.dot(output_grad, w3.T)*gradient(act_layer2)

    w2_grad = np.dot(layer1.T, layer2_grad)
    b2_grad = np.sum(layer2_grad, axis = 0, keepdims=True)
    layer1_grad = np.dot(layer2_grad,w2.T)*gradient(act_layer1)

    w1_grad = np.dot(single_xtrain.T, layer1_grad)
    b1_grad = np.sum(layer1_grad, axis = 0, keepdims = True)

    #Updating the new weights by magnitude of the learning rate
    w3 += learning_rate*w3_grad
    w2 += learning_rate*w2_grad
    w1 += learning_rate*w1_grad

    #Updating the new biases
    b3 += learning_rate*b3_grad
    b2 += learning_rate*b2_grad
    b1 += learning_rate*b1_grad

  err /= samples
  print('epoch %d/%d   error=%f' % (i+1, epochs, err))





epoch 1/2   error=0.038452
epoch 2/2   error=0.032237


In [250]:
from scipy.special import softmax

single_xtest = x_test[0:5]
single_ytest = y_test[0:5]

#Pass through first hidden layer and activation neruon
layer1 = np.dot(single_xtest,w1)+b1
act_layer1 = tanh(layer1)

#Pass through second hidden layer and activation neuron
layer2 = np.dot(act_layer1,w2)+b2
act_layer2 = tanh(layer2)

#Pass through output layer and acrivation neuron
layer3 = np.dot(act_layer2,w3)+b3
output = tanh(layer3)

from scipy.special import softmax
print(mse(single_ytest, output))
print(output)
print(single_ytest)

0.14836153507956196
[[[-1.67528267e-02 -4.46503401e-02  3.23200155e-02  1.45708325e-01
    4.63883332e-03 -2.19556711e-01  9.36257014e-02  8.27359964e-01
   -8.89583267e-02  2.02386661e-01]]

 [[ 3.05674438e-02 -4.86817108e-02  7.90556598e-01  2.01470452e-01
   -1.59790076e-01  1.47472905e-01  1.60399378e-01  5.28219648e-02
   -1.75692760e-01 -5.42858498e-04]]

 [[ 5.12194309e-02  9.05564639e-01 -7.67136727e-02  5.46697093e-02
   -6.78935553e-02  8.46016548e-03  9.54258796e-03  1.13365239e-01
   -5.59008774e-02 -3.45026420e-04]]

 [[ 8.94934669e-01 -8.81280318e-02  4.28645983e-02  5.65515742e-02
   -6.13193182e-02 -2.87296045e-01  2.50617271e-01 -6.83880883e-02
    1.14850392e-01  8.89197868e-02]]

 [[-1.35603962e-01  1.24824763e-01  1.71282932e-01 -2.88977189e-02
    6.11612920e-01 -5.77132729e-03  1.00866207e-01  4.85680896e-03
   -1.10478326e-01  2.87065839e-01]]]
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0